In [1]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name

bucket='sagemaker-intent-classification-training' # Put your s3 bucket name here
prefix = 'input/data' # Used as part of the path in the bucket where you store data
# customize to your bucket where you will store data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)
print(bucket_path)

https://s3-us-east-1.amazonaws.com/sagemaker-sentence-classification


In [2]:
%%sh

# The name of our algorithm
algorithm_name=intent-classification-training

chmod +x src/*

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} -f Dockerfile.cpu .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]

Step 1/7 : FROM phenompeople/centos-python:3.6.3
 ---> e3d7d8ca4a30
Step 2/7 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 64723ce59c04
Step 3/7 : RUN echo ${PATH}
 ---> Using cache
 ---> 3a252e643631
Step 4/7 : ADD requirements.txt /
 ---> Using cache
 ---> 53682fdf2481
Step 5/7 : RUN pip3 install -r requirements.txt
 ---> Using cache
 ---> 8827f8b412de
Step 6/7 : COPY src /opt/program
 ---> Using cache
 ---> 5095a11dfb14
Step 7/7 : WORKDIR /opt/program
 ---> Using cache
 ---> 9e88bbc8432d
Successfully built 9e88bbc8432d
Successfully tagged sentence-classification:latest
The push refers to repository [850021735523.dkr.ecr.us-east-1.amazonaws.com/sentence-classification]
1a304bbdd7ed: Preparing
51b04044f8dc: Preparing
0776cb2696f0: Preparing
952e0784686f: Preparing
65c06ae44bbd: Preparing
f194f1dd3e8f: Preparing
ea264623c568: Preparing
c4cd48200f79: Preparing
bcc97fbfc9e1: Preparing
f194f1dd3e8f: Waiting


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [4]:
data_location = 's3://{}/data'.format(bucket)
print("data location - " + data_location)

output_location = 's3://{}/output'.format(bucket)
print("output location - " + output_location)

data location - s3://sagemaker-sentence-classification/data
output location - s3://sagemaker-sentence-classification/output


In [5]:
import sagemaker as sage
sess = sage.Session()

In [6]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/intent-classification-training'.format(account, region)

In [7]:
classifier = sage.estimator.Estimator(image_name=image, 
                                      role=role,
                                      train_instance_count=1, 
                                      train_instance_type='ml.c5.2xlarge',
                                      output_path=output_location,
                                      sagemaker_session=sess)

In [8]:
classifier.fit(data_location)

2019-06-06 10:07:05 Starting - Starting the training job...
2019-06-06 10:07:06 Starting - Launching requested ML instances......
2019-06-06 10:08:15 Starting - Preparing the instances for training...
2019-06-06 10:08:50 Downloading - Downloading input data
2019-06-06 10:08:50 Training - Downloading the training image...
2019-06-06 10:09:30 Training - Training image download completed. Training in progress...
Using TensorFlow backend.
Script Status - Starting
Reading the data
Exception in reading data: [Errno 2] No such file or directory: '/opt/ml/input/data/training/softskillexpcrossvalidationtrnbert1.txt'
Traceback (most recent call last):
  File "/opt/program/train", line 37, in read_data
    x = open(os.path.join(training_path, path)).readlines()
FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/input/data/training/softskillexpcrossvalidationtrnbert1.txt'

Exception in reading data: [Errno 2] No such file or directory: '/opt/ml/input/data/training/softskillexpcrossva